# Sonde3 
##  Reads and converts binary water quality environmental instrument data to a DataFrame


### I.  Example Usage

Lets dive in!  

We have a example water quality instrument binary file `"tests/ysi_test_files/SA08.dat"`.  This file was generated by a YSI 600LS instrument and is in proprietary binary format.

#### Using the `sonde()` function we:

1.  `autodetect()` the file type and pass to the correct parser function 
2.  `read_ysi()` the binary file and convert to pandas DataFrame
3.  Transform all datetimes to the UTC timezone
4.  Standardize the units to metric and rename the columns to standard name conventions
3.  Pass the DataFrame to `calculate_salinity_psu()` and `calculate_do_mgl()` to apply standard formulas to generate the salinity and dissolved oxygen columns.

In [1]:
import sonde3
import pandas
metadata, df = sonde3.sonde("tests/ysi_test_files/SA08.dat", remove_invalids=False, twdbparams=True)

T:\CoastalScience\Users\ETurner\python\sonde3\sonde3\sonde.py:20: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi(filename, tzinfo)
C:\Users\eturner\.conda\envs\sonde3wdft\lib\site-packages\seawater\library.py:363: RuntimeWarning: invalid value encountered in sqrt
  Rtx = (rt) ** 0.5


#### Why the runtime warnings?

1.  The YSI instrument files don't contain any timezone information.  Therefore, the function has to assume that the timezone of the file to make the UTC conversion.

2. Often raw instrument files will contain impossible & incorrect values in the beginning and end of the file.  Examples: negative values for salinity or dissolved oxygen percentage.  `sonde3` does not trim the raw file, or perform QA analysis.  `sonde3` will pass the values as they were recorded by the instrument.

To automatically convert invalid negative values to zero, either set the flag **remove_invalids=False** or remove it altogether, as this is the default behavior of the package

##### We can now interact with the two dataframes produced by `sonde3`:


In [2]:
df.info() 
df.head() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 9 columns):
Datetime_(UTC)                               700 non-null datetime64[ns, UTC]
water_temperature                            700 non-null float64
water_electrical_conductivity                700 non-null float64
water_depth_nonvented                        700 non-null float64
water_dissolved_oxygen_percent_saturation    700 non-null float64
instrument_battery_voltage                   700 non-null float64
water_specific_conductance                   700 non-null float64
seawater_salinity                            678 non-null float64
water_dissolved_oxygen_concentration         678 non-null float64
dtypes: datetime64[ns, UTC](1), float64(8)
memory usage: 49.3 KB


,Datetime_(UTC),water_temperature,water_electrical_conductivity,water_depth_nonvented,water_dissolved_oxygen_percent_saturation,instrument_battery_voltage,water_specific_conductance,seawater_salinity,water_dissolved_oxygen_concentration
0,2008-07-16 12:00:31+00:00,28.998718,0.000037,0.010862,93.391418,6.09375,0.000035,0.013536,7.183420
1,2008-07-16 13:00:31+00:00,28.482361,0.000059,0.016358,96.765137,6.09375,0.000055,0.013326,7.510631
2,2008-07-16 14:00:31+00:00,27.257385,0.000546,0.017263,103.529358,6.09375,0.000524,0.012655,8.212117
3,2008-07-16 15:00:31+00:00,29.507751,21.301758,0.542648,93.055725,6.09375,19.613107,11.601472,6.655432
4,2008-07-16 16:00:31+00:00,29.762268,21.454102,0.557098,94.188690,6.09375,19.665354,11.631321,6.706414


In [3]:
metadata

,Model,Manufacturer,Instrument_Serial_Number,Station,Deployment_Setup_Time,Filename,Deployment_Start_Time,Deployment_Stop_Time
0,600,YSI,1012,SANT_CDT,,SA08.dat,2008-07-16 12:00:31+00:00,2008-08-14 15:00:16+00:00


In [4]:
len(metadata)

1

### II.  Working with time zones


What if data was collected outside of US/Central time?  Pass the timezone information to `sonde3.sonde`:

In [5]:
import pytz
metadata, df = sonde3.sonde("tests/ysi_test_files/SA08.dat", pytz.timezone('US/Eastern'))
df.head()

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_specific_conductivity_mS/cm,water_salinity_PSU,water_DO_mgl
0,2008-07-16 11:00:31+00:00,28.998718,0.000037,0.010862,93.391418,6.09375,0.000035,0.013536,7.183420
1,2008-07-16 12:00:31+00:00,28.482361,0.000059,0.016358,96.765137,6.09375,0.000055,0.013326,7.510631
2,2008-07-16 13:00:31+00:00,27.257385,0.000546,0.017263,103.529358,6.09375,0.000524,0.012655,8.212117
3,2008-07-16 14:00:31+00:00,29.507751,21.301758,0.542648,93.055725,6.09375,19.613107,11.601472,6.655432
4,2008-07-16 15:00:31+00:00,29.762268,21.454102,0.557098,94.188690,6.09375,19.665354,11.631321,6.706414


### III. Autodetecting files



Curious about what kind of instrument files you have in a directory?  Apply the `sonde3.autodetect` method:

```python
#what kind of file is this??
sonde3.autodetect("tests/greenspan_test_files/RIOA_20060718_CDT_GS7837.xls") 
```

In [6]:
sonde3.autodetect("tests/greenspan_test_files/RIOA_20060718_CDT_GS7837.xls") 

'greenspan_xls'

Copy this code snippet to the notebook line below to run `autodetect()` on all of the use-case examples in the `sonde3` package:

```python
#this script runs through all of the text examples and prints out the autodetect results
import os

root_dir = 'tests'
results = []
for directory, subdirectories, files in os.walk(root_dir):
    for file in files:
        if "_test.txt" in file:
            continue
        os.path.join(directory, file)
        results.append(os.path.join(directory, file) + ' ' + sonde3.autodetect(os.path.join(directory, file)))

results 
```

### IV. Generating Salinity and Dissolved Oxygen

Typically deployed water quality instruments do not compute all rows of data internally.  Instead, these are calculated by the program used to read the file back at the lab.  For example, YSI instruments do not compute salinity or dissolved oxygen concentration.  This can cause confusion because when viewing a YSI *\*.dat* file in YSI's ECOWIN, or Ecowatch Lite program displays salinity and DO mg/L.  However, the raw binary file does not include these rows as they were not physically collected by the instrument during deployment.

For example, lets read the raw binary file of the example file `"tests/ysi_test_files/SA08.dat"` and see what it contains:

In [7]:
metadata, SA08_BIN = sonde3.read_ysi("tests/ysi_test_files/SA08.dat",pytz.timezone('US/Central'))
SA08_BIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 6 columns):
Datetime_(UTC)                700 non-null datetime64[ns, UTC]
water_temp_C                  700 non-null float64
water_conductivity_mS/cm      700 non-null float64
water_depth_m_nonvented       700 non-null float64
water_DO_%                    700 non-null float64
instrument_battery_voltage    700 non-null float64
dtypes: datetime64[ns, UTC](1), float64(5)
memory usage: 32.9 KB


The pandas DataFrame contains the columns for conductivity and % DO saturation, but not for salinity and DO concentration.

For comparision, lets read the comma separated version of this file that was produced by the proprietary YSI Ecowin program:

In [8]:
metadata, SA08_CSV = sonde3.read_ysi_ascii("tests/ysi_test_files/SA08.CDF", pytz.timezone('US/Central'),delim=",")


The ECOWIN exported comma separated file has far more columns! The extra columns were derived through formulas using the instrumnet observed measurments.

To generate these columns pass the SA08_BIN DataFrame to calculate Salinity (PSU) and Dissolved Oxygen (mg/L) using `sonde3.calculate_salinity_psu()` and `sonde3.calculate_do_mgl()`

We can then compare our computed results to the ECOwatch program results:

In [9]:
SA08_BIN = sonde3.calculate_salinity_psu(SA08_BIN)
SA08_BIN = sonde3.calculate_do_mgl(SA08_BIN)
SA08_BIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
Datetime_(UTC)                700 non-null datetime64[ns, UTC]
water_temp_C                  700 non-null float64
water_conductivity_mS/cm      700 non-null float64
water_depth_m_nonvented       700 non-null float64
water_DO_%                    700 non-null float64
instrument_battery_voltage    700 non-null float64
water_salinity_PSU            678 non-null float64
water_DO_mgl                  678 non-null float64
dtypes: datetime64[ns, UTC](1), float64(7)
memory usage: 43.9 KB


So, why does `sonde3` produce less values for salinity and DO than the comma separated file?  Is this a bug?

*NO!*  If you recal from the `sqrt()` warning above, some of the values in the file are invalid.  This produces null (**NaN**) values in those rows.

In [10]:
SA08_BIN.tail()

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_salinity_PSU,water_DO_mgl
695,2008-08-14 11:00:16+00:00,23.977051,-0.000006,-0.052356,-0.006866,5.546875,NaN,NaN
696,2008-08-14 12:00:16+00:00,23.938599,-0.000005,-0.050262,-0.006866,5.468750,NaN,NaN
697,2008-08-14 13:00:16+00:00,24.039307,-0.000005,-0.045875,-0.006866,5.468750,NaN,NaN
698,2008-08-14 14:00:16+00:00,24.107056,-0.000005,-0.038671,-0.006866,5.468750,NaN,NaN
699,2008-08-14 15:00:16+00:00,24.177551,-0.000006,-0.034785,-0.006866,5.468750,NaN,NaN


Lets check our two files to see if our conversion methods are those used by YSI.  

Lets use `numpy.random` to pick a row to check:



In [11]:
from numpy import random
row = random.randint(2,677) # pick random row

sub1 =  SA08_BIN.iloc[row:row+1]  #row one binary file
sub2 = SA08_CSV.iloc[row:row+1]   #row two csv file

pandas.concat([sub1, sub2], axis=0,join="inner")

,Datetime_(UTC),water_temp_C,water_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,instrument_battery_voltage,water_salinity_PSU,water_DO_mgl
354,2008-07-31 06:00:31+00:00,30.043335,34.165039,0.55722,81.759644,5.859375,19.259397,5.556196
354,2008-07-31 06:00:31+00:00,30.040000,34.165000,0.55700,81.800000,5.900000,19.260000,5.560000


Looks the same!  Notice, however, that the csv file values are rounded due to the fixed digit precision.

### V. Files from other Manufacturers and Models
#### a. Hydrotech

In [12]:
#metadata, mydf = sonde3.sonde("tests/hydrotech_test_files/0109DELT.CSV", pytz.timezone('US/Central'))

In [13]:
#metadata

In [14]:
#mydf.info()

In [15]:
#mydf.head()

In [16]:
#mydf.tail()

#### b. YSI EXO Units

YSI EXO units are timezone aware.  Thus, we discover the internal timezone and convert to UTC if applicable.

In [17]:
sonde3.autodetect("tests/ysi_test_files/GE-SA-B_17H104157_090617_060000.csv")

'ysi_exo_csv'

In [18]:
#metadata, exo_df = sonde3.sonde("tests/ysi_test_files/GE-SA-B_17H104157_090617_060000.csv")

In [19]:
metadata

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time
0,YSI,,,,,2008-07-16 12:00:31+00:00,2008-08-14 15:00:16+00:00


In [20]:
#exo_df.head()

#### c. YSI ascii + Comma Separated.

YSI timeseries produced by kermit transfer from YSI handset protocol.

In [21]:
metadata, df = sonde3.sonde("tests/ysi_test_files/0917GEB.txt")

T:\CoastalScience\Users\ETurner\python\sonde3\sonde3\sonde.py:30: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi_ascii(filename, tzinfo, ',', None, [1, 2, 3])


In [22]:
metadata

,Manufacturer,Instrument_Serial_Number,Model,Station,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time
0,YSI,,,,,2017-09-08 12:15:08+00:00,2017-09-26 20:00:08+00:00


In [23]:
df.head()

,Datetime_(UTC),water_temp_C,water_tds_g/L,water_pressure_abs,water_depth_m_nonvented,instrument_battery_voltage,water_conductivity_mS/cm,water_specific_conductivity_mS/cm,water_salinity_PSU
0,2017-09-08 12:15:08+00:00,19.93,0.0,14.763,0.303,6.2,0.0,0.0,0.010285
1,2017-09-08 12:30:08+00:00,20.05,0.0,14.767,0.306,6.2,0.0,0.0,0.010336
2,2017-09-08 12:45:08+00:00,20.54,0.0,14.769,0.307,6.2,0.0,0.0,0.010544
3,2017-09-08 13:00:08+00:00,20.91,0.0,14.771,0.309,6.2,0.0,0.0,0.010699
4,2017-09-08 13:15:08+00:00,21.21,0.0,14.774,0.311,6.2,0.0,0.0,0.010823


#### D. Lowell Tiltmeter Comma Separated.

Concatinated timeseries file from a Lowell tiltmeter

In [24]:
metadata, df = sonde3.sonde("tests/lowell_test_files/1603103_SAB_05122016.csv")

T:\CoastalScience\Users\ETurner\python\sonde3\sonde3\sonde.py:38: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_lowell(filename, tzinfo, ',')


In [25]:
sonde3.merge_lowell()

In [26]:
metadata.head()

,Deployment_Setup_Time,Deployment_Start_Time,Deployment_Stop_Time,Filename,Instrument_Serial_Number,Manufacturer,Model,Station
0,NaN,2016-05-13 17:30:00+00:00,2016-07-15 18:00:00+00:00,NaN,1603103,Lowell,TCM,NaN


#### D. Insitu AquaTROLL Comma Separated.



In [27]:

metadata,CCBAY = sonde3.sonde("tests/aquatroll_test_files/HydroVu_AquaTROLL__Baffin_Bay_-1031427_2023-08-15_14-14-09_Export.csv")
metadata,site_1031413 = sonde3.sonde("tests/aquatroll_test_files/HydroVu_default-1031413_2023-10-10_14-52-24_Export.csv", twdbparams=True)


C:\Users\eturner\.conda\envs\sonde3wdft\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [28]:
site_1031413.head()

,Datetime_(UTC),water_temperature,chlorophyll_a_rfu,water_ph,water_dissolved_oxygen_percent_saturation,water_pressure_abs,water_depth_nonvented,water_specific_conductance,water_electrical_conductivity,seawater_salinity,water_dissolved_oxygen_concentration
0,2023-10-10 15:00:00,24.721664,0.040888,8.292933,93.163841,-14.252751,-0.0,55.137098,54.843977,36.541408,6.284414
1,2023-10-10 15:15:00,24.708616,0.046670,8.292661,94.330620,-14.251598,-0.0,57.179242,56.861015,38.072856,6.309232
2,2023-10-10 15:30:00,24.696325,0.045111,8.294389,94.212906,-14.218081,-0.0,34.556047,34.355615,21.705158,6.918864
3,2023-10-10 15:45:00,24.695467,0.148413,8.300394,95.430084,-14.250790,-0.0,54.745043,54.426614,36.249236,6.450886
4,2023-10-10 16:00:00,24.704952,0.090938,8.297940,95.449501,-14.222909,-0.0,62.216926,61.866308,41.895015,6.246908


In [29]:
CCBAY.head()

,Datetime_(UTC),water_salinity_PSU,water_ph,water_DO_%,water_temp_C,water_DO_mgl
0,2023-08-15 21:32:00,0.0,9.485452,114.515373,34.295311,8.051721
1,2023-08-15 21:45:00,0.0,10.781005,113.995239,33.333256,8.144041
2,2023-08-15 22:00:00,0.0,10.802633,113.062195,32.057720,8.252015
3,2023-08-15 22:15:00,0.0,5.509067,111.411888,30.864656,8.298081
4,2023-08-15 22:30:00,0.0,7.524588,110.970390,29.944023,8.397066


### VI. Package Validation Tests

#### A.  Check to see if the package is correctly handling daylight savings time.

The test case dataset **tests/ysi_test_files/0108BAYT.csv** was collected during a daylight savings transition.  These three files were processed differently at the time of instrument processing.  The *.dat* file is the raw binary file from the instrument.  The *CDF* file was produced by the Ecowatch program.  The *cvs* was produced by converting the *CDF* file in ms excel to a comma separated file.  If the times were not consistant between the three files it would indicate **sonde3** is not processing datetime correctly.

In [30]:
#import all three versions of the file ,CDF, csv, and binary *.dat
metadata, baytcsv = sonde3.sonde("tests/ysi_test_files/0108BAYT.csv", pytz.timezone('US/Central'))
metadata, baytdat = sonde3.sonde("tests/ysi_test_files/0108BAYT.dat", pytz.timezone('US/Central'))
metadata, baytCDF = sonde3.sonde("tests/ysi_test_files/0108BAYT.CDF", pytz.timezone('US/Central'))

In [31]:
#cut the first & last row from each file.
pandas.concat([baytCDF.iloc[0:1], baytdat.iloc[0:1], baytcsv.iloc[0:1] , \
               baytCDF.iloc[-1:], baytdat.iloc[-1:], baytcsv.iloc[-1:]], axis=0,join="inner")

,Datetime_(UTC),water_temp_C,water_specific_conductivity_mS/cm,water_depth_m_nonvented,water_DO_%,water_conductivity_mS/cm,water_DO_mgl,water_salinity_PSU
0,2008-01-29 15:00:33+00:00,22.560000,-0.0,-0.0,103.200000,-0.0,8.927547,0.011370
0,2008-01-29 15:00:33+00:00,22.562561,-0.0,-0.0,103.227234,-0.0,8.929467,0.011371
0,2008-01-29 15:00:00+00:00,22.560000,0.0,-0.0,103.200000,0.0,8.927547,0.011370
1061,2008-03-13 19:00:33+00:00,22.510000,-0.0,-0.0,104.300000,-0.0,9.031326,0.011350
1061,2008-03-13 19:00:33+00:00,22.510376,-0.0,-0.0,104.284668,-0.0,9.029934,0.011350
1061,2008-03-13 19:00:00+00:00,22.510000,0.0,-0.0,104.300000,0.0,9.031326,0.011350


#### B.  More parsing testing

In [32]:
metadata, df = sonde3.sonde("tests/lowell_test_files/1708018_3Basin.csv", twdbparams=True)
df.columns

Index(['Datetime_(UTC)', 'water_speed', 'water_bearing',
       'northward_water_velocity', 'eastward_water_velocity',
       'water_temperature'],
      dtype='object')

In [33]:
metadata, df = sonde3.sonde("tests/ysi_test_files/spotchecks-galveston-180618-081639.csv", twdbparams=True)
df.columns

T:\CoastalScience\Users\ETurner\python\sonde3\sonde3\sonde.py:26: UserWarning: Info: No time zone was set for file, assuming records are recorded in CST
  metadata, df = formats.read_ysi_exo_backup(filename)


Index(['Datetime_(UTC)', 'water_temperature', 'mmHg',
       'water_dissolved_oxygen_percent_saturation',
       'water_dissolved_oxygen_concentration', 'water_ph', 'water_bga_pe_rfu',
       'chlorophyll_a_rfu', 'water_depth_nonvented',
       'instrument_battery_voltage', 'ALTm', 'Lat', 'Lon',
       'water_specific_conductance', 'water_electrical_conductivity',
       'seawater_salinity'],
      dtype='object')

In [34]:
metadata, df = sonde3.sonde("tests/ysi_test_files/EMBData_Retrieved10.3.2019.csv")
df.head(10)

,Datetime_(Native),Time (Fract. Sec),Site Name,Cond µS/cm,Depth m,nLF Cond µS/cm,Pressure psi a,Sal psu,SpCond µS/cm,TDS mg/L,Wiper Position volt,Temp °C,Vertical Position m,Battery V,Cable Pwr V
0,2019-08-28 10:30:00,0.0,MATLAB,0.1,0.054,0.1,0.077,0.00,0.1,0,1.222,28.329,0.117,3.10,1.7
1,2019-08-28 11:00:00,0.0,MATLAB,0.4,0.132,0.4,0.188,0.00,0.4,0,1.219,28.852,0.199,3.09,1.7
2,2019-08-28 11:30:00,0.0,MATLAB,0.0,0.115,0.0,0.163,0.00,0.0,0,1.219,29.873,0.183,3.11,1.7
3,2019-08-28 12:00:00,0.0,MATLAB,39362.5,0.362,35206.4,0.523,22.39,35623.8,23155,1.228,30.483,0.430,3.13,1.7
4,2019-08-28 12:30:00,0.0,MATLAB,40608.4,0.370,36254.8,0.533,23.06,36692.6,23850,1.228,30.587,0.435,3.13,1.7
5,2019-08-28 13:00:00,0.0,MATLAB,40809.3,0.361,36237.2,0.520,23.06,36698.3,23854,1.219,30.865,0.425,3.13,1.7
6,2019-08-28 13:30:00,0.0,MATLAB,40882.7,0.357,36312.4,0.514,23.11,36773.2,23903,1.219,30.851,0.422,3.13,1.7
7,2019-08-28 14:00:00,0.0,MATLAB,40925.7,0.339,36414.8,0.488,23.18,36869.2,23965,1.215,30.760,0.401,3.13,1.7
8,2019-08-28 14:30:00,0.0,MATLAB,40870.3,0.330,36410.1,0.475,23.18,36859.1,23958,1.215,30.698,0.394,3.13,1.7
9,2019-08-28 15:00:00,0.0,MATLAB,40838.0,0.325,36383.6,0.468,23.16,36832.0,23941,1.223,30.695,0.389,3.13,1.7


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775 entries, 0 to 1774
Data columns (total 15 columns):
Datetime_(Native)      1775 non-null datetime64[ns]
Time (Fract. Sec)      1775 non-null float64
Site Name              1775 non-null object
Cond µS/cm             1775 non-null float64
Depth m                1775 non-null float64
nLF Cond µS/cm         1775 non-null float64
Pressure psi a         1775 non-null float64
Sal psu                1775 non-null float64
SpCond µS/cm           1775 non-null float64
TDS mg/L               1775 non-null int64
Wiper Position volt    1775 non-null float64
Temp °C                1775 non-null float64
Vertical Position m    1775 non-null float64
Battery V              1775 non-null float64
Cable Pwr V            1775 non-null float64
dtypes: datetime64[ns](1), float64(12), int64(1), object(1)
memory usage: 208.1+ KB


#### C.  Testing library functionality

This should ideally match our WDFT production servers.
(env) wdft@ip-10-40-0-138:/var/www/coastal/env/bin$ python version_check.py
Python: 3.6.8 (default, Oct 7 2019, 12:59:55)
[GCC 8.3.0]
Our Maxsize: 9223372036854775807
pandas: 0.25.3
seawater: 3.3.4
six: 1.13.0
csv: 1.0
pytz: 2019.3




In [36]:

import pandas
import seawater
import datetime
import pytz
import sys
import csv
import six
import pytest
import sonde3
print ("Sonde3: ", sonde3.__version__)
print ("pytest: ", pytest.__version__)
print ("Python: ", sys.version)
print ("Our Maxsize: ", sys.maxsize)
print ("pandas: ", pandas.__version__)
print ("seawater: ", seawater.__version__)
print ("six: ", six.__version__)
print ("csv: ", csv.__version__)
print ("pytz: ", pytz.__version__)

Sonde3:  3.11
pytest:  7.0.1
Python:  3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Our Maxsize:  9223372036854775807
pandas:  0.25.3
seawater:  3.3.4
six:  1.16.0
csv:  1.0
pytz:  2023.3.post1


In [37]:
import io, itertools


In [38]:
DF

NameError: name 'DF' is not defined

In [ ]:
DF